In [ ]:
import pandas as pd
import pickle
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.models import FastText
import numpy as np
import csv
from sklearn.feature_extraction.text import CountVectorizer
from mittens import Mittens

## One Hot Encoding

In [ ]:
def one_hot_dictionary(vocab):
  length = len(vocab)
  onehotdict = {}
  vocab = list(vocab)
  for i in range(length):
    temp = [0]*length
    temp[i] = 1
    onehotdict[vocab[i]] = temp
  return onehotdict

In [ ]:
ohd = one_hot_dictionary(vocab)
f = open("one_hot.pkl","wb")
pickle.dump(ohd,f)
f.close()

## Bag of Words

In [ ]:
def BoW(vocab_sentence):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(vocab_sentence)
    df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())
    d = df_bow_sklearn.to_dict()
    bow = {}
    for i in df_bow_sklearn:
        bow[i] =(list(d[i].values()))
    return bow
    

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
b = BoW(vocab_sentence)
f = open("BoW.pkl","wb")
pickle.dump(b,f)
f.close()

## TFIDF(Text Frequency Inverse Document Frequency)

In [ ]:
def TFIDF(vocab_sentence):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(vocab_sentence)
    df_tfidf_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
    d = df_tfidf_sklearn.to_dict()
    tfidf = {}
    for i in df_tfidf_sklearn:
        tfidf[i] = (list(d[i].values()))
    return tfidf

In [ ]:
t = TFIDF(vocab_sentence)
f = open("TfIdf.pkl","wb")
pickle.dump(t,f)
f.close()

## word2vec - Continuous Bag of Words

In [ ]:
def cbow_fun(vocab_sentence):
    cbow = Word2Vec([x.split(' ') for x in vocab_sentence], vector_size=100, window=5, min_count=1, workers=4, sg = 10, cbow_mean = 1)
    cbow_dict = {}
    for i in cbow.wv.key_to_index.keys():
        cbow_dict[i] = (cbow.wv[i])
    return cbow_dict

In [ ]:
cbow = cbow_fun(vocab_sentence)
f = open("CBoW.pkl","wb")
pickle.dump(cbow,f)
f.close()

## word2vec - Skipgram

In [ ]:
def skipgram_fun(vocab_sentence):
    skipgram = Word2Vec([x.split(' ') for x in vocab_sentence], vector_size=100, window=5, min_count=1, workers=4, sg = 1)
    sg_dict = {}
    for i in skipgram.wv.key_to_index.keys():
        sg_dict[i] = (skipgram.wv[i])
    return sg_dict

In [ ]:
s = skipgram_fun(vocab_sentence)
f = open("Skipgram.pkl","wb")
pickle.dump(s,f)
f.close()

## Fasttext

In [ ]:
def fasttext(vocab_sentence):
    ft = FastText([x.split(' ') for x in vocab_sentence], vector_size=100, window=5, min_count=1, epochs = 10)
    ft_dict = {}
    for i in ft.wv.key_to_index.keys():
        ft_dict[i] = (ft.wv[i])
    return ft_dict

In [ ]:
ft = fasttext(vocab_sentence)
f = open("fasttext.pkl","wb")
pickle.dump(ft,f)
f.close()

## Glove

In [ ]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed
glove_path = "glove.twitter.27B.100d.txt"
pre_glove = glove2dict(glove_path)

In [ ]:
cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(doc) #doc has words in the document that does not belong to Glove
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

In [ ]:
# Fine-tuning pre-trained glove
mittens_model = Mittens(n=100, max_iter=1000)
new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

In [ ]:
newglove = dict(zip(corp_vocab, new_embeddings))
f = open("Glove.pkl","wb")
pickle.dump(newglove,f)
f.close()